# Books.txt Dosyasına Ulaşmak İçin Drive Dosyalarının Projeye Aktarılması

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Proje Kodları

In [ ]:
import random
import datetime

class User:
    def __init__(self, username=None, password=None):
        self.path_to_books = "/content/drive/MyDrive/books.txt"
        self.username = username
        self.password = password
        self.book_collection_file = f"{self.username}_collection.txt" if self.username else None

    def add_book_to_collection(self):
        book_info = None  # Kitap bilgilerini saklamak için bir değişken
        try:
            if self.username:
                title = input("Enter the title of the book you want to add to your collection: ").strip().lower()

                # Kitabı books.txt dosyasında ara
                found = False
                with open(self.path_to_books, "r") as file:
                    for line in file:
                        book_title = line.split(",")[0].strip().lower()
                        if book_title == title:
                            found = True
                            book_info = line.strip()
                            break

                if found:
                    # Koleksiyonda olan bir kitap ise koleksiyona ekle
                    with open(self.book_collection_file, "a+") as collection_file:
                        collection_file.seek(0)
                        if book_info in collection_file.read():
                            print("This book is already in your collection.")
                        else:
                            collection_file.write(book_info + "\n")
                            print("Book added to your collection successfully.")
                else:
                    print("The book is not available in the library.")
            else:
                print("Please login first.")
        except FileNotFoundError:
            print("Error: Collection file not found.")
        except Exception as e:
            print(f"Error: Unable to add book to collection - {e}")

    def remove_book_from_collection(self, title):
        try:
            if self.username:
                with open(self.book_collection_file, "r") as file:
                    lines = file.readlines()
                with open(self.book_collection_file, "w") as file:
                    for line in lines:
                        if not line.startswith(title):
                            file.write(line)
                    print("Book removed from your collection successfully.")
            else:
                print("Please login first.")
        except FileNotFoundError:
            print("Error: Collection file not found.")

    def list_books_in_collection(self):
        try:
            if self.username:
                with open(self.book_collection_file, "r") as file:
                    books = file.readlines()
                    if not books:
                        print("No books found in your collection.")
                    else:
                        print("Listing books in your collection:")
                        for index, book in enumerate(books, start=1):
                            book_info = book.strip().split(",")
                            print(f"{index}. Book Title: {book_info[0]}, Author: {book_info[1]}")
            else:
                print("Please login first.")
        except FileNotFoundError:
            print("You don't have any books in your collection yet.")


class Library:
    def __init__(self):
        try:
            self.file_name = '/content/drive/MyDrive/books.txt'
            self.users = {}  # Kullanıcı adı -> User nesnesi eşleştirmesi için bir sözlük
            self.logged_in_user = None  # Oturum açmış kullanıcı
            self.load_users()  # Kullanıcıları dosyadan yükle
            self.file = open(self.file_name, "a+") # Kitap dosyasını açıyoruz
        except Exception as e:
            print(f"Error: Unable to initialize the library - {e}")

    # Nesne silindiğinde dosyayı kapatıyoruz
    def __del__(self):
        try:
            self.file.close()
        except Exception as e:
            print(f"Error: Unable to close the file - {e}")

    # Kitapları listeleme fonksiyonu.
    def list_books(self):
        try:
            self.file.seek(0)
            books = self.file.readlines()
            if not books:
                print("No books found in the library.")
            else:
                print("Listing books:")
                for index, book in enumerate(books, start=1):
                    book_info = book.strip().split(",")
                    print(f"{index}. Book Title: {book_info[0]}, Author: {book_info[1]}")
        except Exception as e:
            print(f"Error: Unable to list books - {e}")

     # Kullanıcıdan kitap bilgilerini alarak, kütüphaneye kitap ekleyen fonksiyon.
    def add_book(self):
        try:
            title = input("Enter book title: ")
            author = input("Enter book author: ")
            release_year = input("Enter release year: ")
            pages = input("Enter number of pages: ")
            book_info = f"{title},{author},{release_year},{pages}"
            self.file.write("\n")
            self.file.write(book_info)
            print("Book added successfully.")
        except Exception as e:
            print(f"Error: Unable to add book - {e}")

     # Kütüphaneden kitap kaldırma fonksiyonu
    def remove_book(self):
        try:
            title = input("Enter the title of the book you want to remove: ")
            self.file.seek(0)
            books = self.file.readlines()
            updated_books = []
            found = False
            for book in books:
                book_info = book.strip().split(",")
                if book_info[0] == title:
                    found = True
                else:
                    updated_books.append(book)
            if not found:
                print("Book not found.")
                return
            self.file.seek(0)
            self.file.truncate(0)
            self.file.writelines(updated_books)
            print("Book removed successfully.")
        except Exception as e:
            print(f"Error: Unable to remove book - {e}")

    # Kitapları filtreleme fonksiyonu
    def filter_books(self):
        try:
            print("\nFilter Options:")
            print("1. Filter by Book Title")
            print("2. Filter by Author")
            print("3. Filter by Release Year")
            print("4. Filter by Number of Pages")
            print("5. Sort Books")
            choice = input("Enter your filter option: ")

            if choice == "1":
                self.filter_by_book_title()
            elif choice == "2":
                self.filter_by_author()
            elif choice == "3":
                self.filter_by_release_year()
            elif choice == "4":
                self.filter_by_number_of_pages()
            elif choice == "5":
                self.sort_books()
            else:
                print("Invalid choice. Please try again.")
        except Exception as e:
            print(f"Error: Unable to filter books - {e}")

    # Kitap başlığına göre filtreleme yapmak için özelleştirilmiş fonksiyon
    def filter_by_book_title(self):
        try:
            title = input("Enter book title: ").lower()
            self.file.seek(0)
            books = self.file.readlines()
            choiceTwo = "0"
            print("\nFilter Options:")
            print(f'1. Matching Books with "{title}"')
            print(f'2. Exact Match Books with "{title}"')
            print(f'3. Non-Matching Books with "{title}"')
            choiceTwo = input("Enter your filter option(1-3): ")
            matching_books = []
            exact_match = []
            non_matching_books = []

            for book in books:
                book_info = book.strip().split(",")
                if title in book_info[0].lower():
                    matching_books.append(book)
                    if title == book_info[0].lower():
                        exact_match.append(book)
                else:
                    non_matching_books.append(book)

            if not matching_books:
                print("No books found matching the filter criteria.")
            else:
              if choiceTwo == "1":
                print("Matching Books:")
                if matching_books:
                  for index, book in enumerate(matching_books, start=1):
                      book_info = book.strip().split(",")
                      print(f"{index}. Book Title: {book_info[0]}, Author: {book_info[1]}")
                else:
                  print("No matching books found.")
              elif choiceTwo == "2":
                print("\nExact Match:")
                if exact_match:
                  for index, book in enumerate(exact_match, start=1):
                      book_info = book.strip().split(",")
                      print(f"{index}. Book Title: {book_info[0]}, Author: {book_info[1]}")
                else:
                  print("No exact match found.")
              elif choiceTwo == "3":
                print("\nNon-Matching Books:")
                if non_matching_books:
                  for index, book in enumerate(non_matching_books, start=1):
                      book_info = book.strip().split(",")
                      print(f"{index}. Book Title: {book_info[0]}, Author: {book_info[1]}")
                else:
                  print("No non-matching books found.")
              else:
                print("Invalid choice. Please try again.")

        except Exception as e:
            print(f"Error: Unable to filter books by title - {e}")

     # Yazarına göre filtreleme yapmak için özelleştirilmiş fonksiyon
    def filter_by_author(self):
        try:
            author = input("Enter author: ").lower()
            self.file.seek(0)
            books = self.file.readlines()
            choiceTwo = "0"
            print("\nFilter Options:")
            print(f'1. Matching Books with "{author}"')
            print(f'2. Exact Match Books with "{author}"')
            print(f'3. Non-Matching Books with "{author}"')
            choiceTwo = input("Enter your filter option(1-3): ")

            matching_books = []
            exact_match = []
            non_matching_books = []

            for book in books:
                book_info = book.strip().split(",")
                if author in book_info[1].lower():
                    matching_books.append(book)
                    if author == book_info[1].lower():
                        exact_match.append(book)
                else:
                    non_matching_books.append(book)

            if not matching_books and not non_matching_books:
                print("No books found matching the filter criteria.")
            else:
              if choiceTwo == "1":
                print("Matching Books:")
                if matching_books:
                  for index, book in enumerate(matching_books, start=1):
                      book_info = book.strip().split(",")
                      print(f"{index}. Book Title: {book_info[0]}, Author: {book_info[1]}")
                else:
                  print("No matching books found.")
              elif choiceTwo == "2":
                print("\nExact Match:")
                if exact_match:
                  for index, book in enumerate(exact_match, start=1):
                      book_info = book.strip().split(",")
                      print(f"{index}. Book Title: {book_info[0]}, Author: {book_info[1]}")
              elif choiceTwo == "3":
                print("\nNon-Matching Books:")
                if non_matching_books:
                  for index, book in enumerate(non_matching_books, start=1):
                      book_info = book.strip().split(",")
                      print(f"{index}. Book Title: {book_info[0]}, Author: {book_info[1]}")
                else:
                  print("No non-matching books found.")
              else:
                print("Invalid choice. Please try again.")

        except Exception as e:
            print(f"Error: Unable to filter books by author - {e}")

     # Yayın yılına göre filtreleme yapmak için özelleştirilmiş fonksiyon
    def filter_by_release_year(self):
      try:
        year = int(input("Enter release year: "))
        current_year = datetime.datetime.now().year
        if year > current_year:
            print("Invalid year. Please enter a valid release year.")
            return
        self.file.seek(0)
        books = self.file.readlines()
        filtered_books = []

        choiceTwo = "0"
        print("\nFilter Options:")
        print(f'1. Books published in "{year}"')
        print(f'2. Books published in "{year}" and AFTER')
        print(f'3. Books published in "{year}" and BEFORE')
        print(f'4. Books published between "{year}" and "second year"')
        choiceTwo = input("Enter your filter option(1-4): ")
        if choiceTwo == "4":
            second_year = int(input("Enter second release year: "))
            if second_year > current_year:
                print("Invalid second year. Please enter a valid release year.")
        if second_year<year:
          temp = year
          year = second_year
          second_year = temp

        for book in books:
            book_info = book.strip().split(",")
            book_year = int(book_info[2])
            if choiceTwo=="1" and book_year == year:
                filtered_books.append(book)
            elif choiceTwo=="2" and book_year >= year:  # Option 2
                filtered_books.append(f"{book.strip()}")
            elif choiceTwo=="3 " and book_year <= year:  # Option 3
                filtered_books.append(f"{book.strip()}")
            elif choiceTwo == "4" and year <= book_year <= second_year:
                filtered_books.append(f"{book.strip()}")

        if not filtered_books:
            print("No books found matching the filter criteria.")
        else:
            print("Filtered Books:")
            for index, book in enumerate(filtered_books, start=1):
                print(f"{index}. {book}")

      except ValueError:
        print("Invalid input. Please enter a valid year.")
      except Exception as e:
        print(f"Error: Unable to filter books by release year - {e}")

    # Sayfa sayısına göre filtreleme yapmak için özelleştirilmiş fonksiyon
    def filter_by_number_of_pages(self):
      try:
        pages = input("Enter number of pages: ")
        if not pages.isdigit():
            raise ValueError("Invalid input. Please enter a valid number of pages.")

        pages = int(pages)
        self.file.seek(0)
        books = self.file.readlines()
        filtered_books = []

        choiceTwo = "0"
        print("\nFilter Options:")
        print(f'1. Books with "{pages}" pages')
        print(f'2. Books with "{pages}" pages or MORE')
        print(f'3. Books with "{pages}" pages or LESS')
        print(f'4. Books with "{pages}" pages and "second pages"')
        choiceTwo = input("Enter your filter option(1-3): ")

        if choiceTwo not in ["1", "2", "3", "4"]:
            print("Invalid filter option. Please enter a number between 1 and 3.")
            return

        if choiceTwo == "4":
            second_pages = int(input("Enter value of second pages number: "))
        if second_pages<pages:
          temp = pages
          pages = second_pages
          second_pages = temp

        for book in books:
            book_info = book.strip().split(",")
            book_pages = int(book_info[3])
            if choiceTwo == "1" and book_pages == pages:
                filtered_books.append(book)
            elif choiceTwo == "2" and book_pages >= pages:  # Option 2
                filtered_books.append(f"{book.strip()}")
            elif choiceTwo == "3" and book_pages <= pages:  # Option 3
                filtered_books.append(f"{book.strip()}")
            elif choiceTwo == "4" and pages <= book_pages <= second_pages:
                filtered_books.append(f"{book.strip()}")

        if not filtered_books:
            print("No books found matching the filter criteria.")
        else:
            print("Filtered Books:")
            for index, book in enumerate(filtered_books, start=1):
                print(f"{index}. {book}")

      except ValueError as ve:
        print(ve)
      except Exception as e:
        print(f"Error: Unable to filter books by number of pages - {e}")



    # Kitapları farklı şekillerde sıralamak için fonksiyon
    def sort_books(self):
      try:
        print("\nSort Options:")
        print("1. Sort by Book Title (Alphabetical - A to Z)")
        print("2. Sort by Book Title (Alphabetical - Z to A)")
        print("3. Sort by Author (Alphabetical - A to Z)")
        print("4. Sort by Author (Alphabetical - Z to A)")
        print("5. Sort by Release Year (Ascending)")
        print("6. Sort by Release Year (Descending)")
        print("7. Sort by Number of Pages (Ascending)")
        print("8. Sort by Number of Pages (Descending)")
        choice = input("Enter your sort option: ")
        self.file.seek(0)
        books = self.file.readlines()
        if choice == "1":
            books.sort(key=lambda x: x.strip().split(",")[0].lower())
        elif choice == "2":
            books.sort(key=lambda x: x.strip().split(",")[0].lower(), reverse=True)
        elif choice == "3":
            books.sort(key=lambda x: x.strip().split(",")[1].lower())
        elif choice == "4":
            books.sort(key=lambda x: x.strip().split(",")[1].lower(), reverse=True)
        elif choice == "5":
            books.sort(key=lambda x: int(x.strip().split(",")[2]))
        elif choice == "6":
            books.sort(key=lambda x: int(x.strip().split(",")[2]), reverse=True)
        elif choice == "7":
            books.sort(key=lambda x: int(x.strip().split(",")[3]))
        elif choice == "8":
            books.sort(key=lambda x: int(x.strip().split(",")[3]), reverse=True)
        else:
            print("Invalid choice. Please try again.")
            return
        print("Sorted Books:")
        for index, book in enumerate(books, start=1):
            book_info = book.strip().split(",")
            print(f"{index}. Book Title: {book_info[0]}, Author: {book_info[1]}, Release Year: {book_info[2]}, Pages: {book_info[3]}")
      except Exception as e:
        print(f"Error: Unable to sort books - {e}")


     # Books.txt dosyasından rastgele bir kitap önermek için fonksiyon
    def suggest_random_book(self):
        try:
            self.file.seek(0)
            books = self.file.readlines()
            if not books:
                print("No books available.")
                return

            random_book = random.choice(books)
            book_info = random_book.strip().split(",")
            print("Random Book Suggestion:")
            print(f"Title: {book_info[0]}")
            print(f"Author: {book_info[1]}")
            print(f"Release Year: {book_info[2]}")
            print(f"Number of Pages: {book_info[3]}")
        except Exception as e:
            print(f"Error: Unable to suggest random book - {e}")

    # Verilerden basit analizler çıkarmak için fonksiyon
    def analyze_books(self):
        try:
            self.file.seek(0)
            books = self.file.readlines()
            if not books:
                print("No books available for analysis.")
                return

            print("\nAnalysis Options:")
            print("1. Author Analysis")
            print("2. Yearly Book Count Analysis")
            print("3. Page Count and Year Relationship Analysis")
            analysis_choice = input("Enter your analysis option (1-3): ")

            if analysis_choice == "1":
                # Yazar analizi
                author_count = {}
                for book in books:
                    author = book.strip().split(",")[1]
                    author_count[author] = author_count.get(author, 0) + 1

                print("\nAuthor Analysis:")
                for author, count in author_count.items():
                    print(f"{author}: {count} books")

                    print("#" * count * 10)  #  '#' sembolü ile görselleştirme

            elif analysis_choice == "2":
                # Yıl analizi
                year_count = {}
                for book in books:
                    year = int(book.strip().split(",")[2])
                    year_count[year] = year_count.get(year, 0) + 1

                print("\nYearly Book Count Analysis:")
                for year, count in sorted(year_count.items()):
                    print(f"{year}: {count} books")

                    print("#" * count * 10)  #  '#' sembolü ile görselleştirme

            elif analysis_choice == "3":
                # Sayfa sayısı ve yıl ilişkisi analizi
                page_year_relation = {}
                for book in books:
                    year = int(book.strip().split(",")[2])
                    pages = int(book.strip().split(",")[3])
                    if year in page_year_relation:
                        page_year_relation[year].append(pages)
                    else:
                        page_year_relation[year] = [pages]

                print("\nPage Count and Year Relationship Analysis:")
                for year, pages_list in sorted(page_year_relation.items()):
                    avg_pages = sum(pages_list) / len(pages_list)
                    print(f"In {year}, average page count: {avg_pages:.2f}")

                    # ASCII art ile görselleştirme
                    print("#" * int(avg_pages/10))  #  '#' sembolü ile görselleştirme

            else:
                print("Invalid choice. Please try again.")
                return

        except Exception as e:
            print(f"Error: Unable to analyze books - {e}")

    def load_users(self):
        # Kullanıcıları dosyadan yükle
        try:
            with open("users.txt", "r") as file:
                for line in file:
                    username, password = line.strip().split(",")
                    self.users[username] = User(username, password)
        except FileNotFoundError:
            # Dosya bulunamazsa yeni bir dosya oluşturulacak
            with open("users.txt", "w") as file:
                pass
        except Exception as e:
            print(f"Error loading users: {e}")

    def save_users(self):
        # Kullanıcıları dosyaya kaydet
        with open("users.txt", "w") as file:
            for user in self.users.values():
                file.write(f"{user.username},{user.password}\n")

    def register(self):
        username = input("Enter username: ")
        if username in self.users:
            print("Username already exists. Please choose another one.")
            return
        password = input("Enter password: ")
        self.users[username] = User(username, password)
        self.save_users()  # Kullanıcı bilgilerini dosyaya kaydet
        print("Registration successful.")


    def login(self):
        username = input("Enter username: ")
        password = input("Enter password: ")
        if username in self.users and self.users[username].password == password:
            print("Login successful.")
            self.logged_in_user = self.users[username]
            return self.users[username]  # Kullanıcı nesnesini döndür
        else:
            print("Invalid username or password.")
            return None  # Oturum açılamazsa None döndür

    def logout(self):
        if self.logged_in_user:  # Oturum açıkken
            self.logged_in_user = None
            print("Logged out successfully.")
        else:
            print("No active session.")  # Oturum açık değilse


    def suggest_personalized_book(self):
      try:
        if not self.logged_in_user:
            print("Please login first.")
            return

        # Kullanıcının koleksiyon dosyasını aç
        user_collection_file = f"{self.logged_in_user.username}_collection.txt"
        with open(user_collection_file, "r") as file:
            user_collection = file.readlines()

        if user_collection:
            # Kullanıcının koleksiyonundan rastgele bir kitap seç
            selected_book = random.choice(user_collection)
            selected_author = selected_book.split(",")[1].strip().lower()

            # Seçilen kitabın yazarının diğer kitaplarını ara
            suggested_books = []
            with open(self.file_name, "r") as file:
                for line in file:
                    book_info = line.strip().split(",")
                    if selected_author in book_info[1].strip().lower() and line not in user_collection:
                        suggested_books.append(line)

            if suggested_books:
                # Kullanıcının koleksiyonunda olmayan ve yazarının diğer kitapları arasından rastgele bir kitap öner
                suggested_book = random.choice(suggested_books)
            else:
                # Kullanıcının koleksiyonunda olmayan kitap bulunamazsa, yıl bazında öneri yap
                suggested_books = []
                for line in open(self.file_name, "r"):
                    book_info = line.strip().split(",")
                    if book_info[2].strip() == selected_book.split(",")[2].strip() and line not in user_collection:
                        suggested_books.append(line)

                suggested_book = random.choice(suggested_books)

        else:
            # Kullanıcının koleksiyonunda kitap yoksa, koleksiyonda olmayan bir kitap seç
            with open(self.file_name, "r") as file:
                all_books = file.readlines()
            suggested_book = random.choice(all_books)

            # Kullanıcının koleksiyonunda varsa, tekrar başka bir kitap seç
            while suggested_book in user_collection:
                suggested_book = random.choice(all_books)

        # Önerilen kitabın bilgilerini yazdır
        book_info = suggested_book.strip().split(",")
        print("Personalized Book Suggestion:")
        print(f"Title: {book_info[0]}")
        print(f"Author: {book_info[1]}")
        print(f"Release Year: {book_info[2]}")
        print(f"Number of Pages: {book_info[3]}")

      except Exception as e:
        print(f"Error: Unable to suggest personalized book - {e}")


'''
    # Kullanıcın kendi listesine kitap eklemek için fonksiyon
    def add_book_to_collection(self):
        try:
            if not self.logged_in_user:
                print("Please login first.")
                return

            title = input("Enter the title of the book you want to add to your collection: ").strip().lower()
            found = False

            with open(self.file_name, "r") as file:
                for line in file:
                    book_title = line.split(",")[0].strip().lower()
                    if book_title == title:
                        found = True
                        self.logged_in_user.book_collection.append(line)
                        print("Book added to your collection successfully.")
                        break

            if not found:
                print("The book is not available in the library.")

        except Exception as e:
            print(f"Error: Unable to add book to collection - {e}")
'''
def main():
    lib = Library()
    user = None  # Başlangıçta kullanıcı oturumu açılmamış

    while True:
        print("\n*** MENU ***")
        print("\n*** Main options ***")
        print("1) List Books")
        print("2) Add Book")
        print("3) Remove Book")
        print("4) Filter Books")
        print("5) Analyze Books")
        print("6) I'm Feeling Lucky")
        print("*** Login and registration ***")
        print("7) Register")
        print("8) Login")
        print("*** Special Options for Registered Users ***")
        print("9) Add books to user's private collection")
        print("10) Remove books from user's private collection")
        print("11) List books in user's private collection")
        print("12) Suggest Personalized Book")
        print("13) Logout")
        print("q) Exit")

        choice = input("Enter your choice: ")

        if choice == "1":
            lib.list_books()
        elif choice == "2":
            lib.add_book()
        elif choice == "3":
            lib.remove_book()
        elif choice == "4":
            lib.filter_books()
        elif choice == "5":
            lib.analyze_books()
        elif choice == "6":
            lib.suggest_random_book()
        elif choice == "7":
            lib.register()
        elif choice == "8":
            user = lib.login()  # Kullanıcı oturumu açıldı
        elif choice == "9":
            if user:
                user.add_book_to_collection()
            else:
                print("Please login first.")
        elif choice == "10":
            if user:
                title = input("Enter the title of the book you want to remove: ")
                user.remove_book_from_collection(title)
            else:
                print("Please login first.")
        elif choice == "11":
            if user:
                user.list_books_in_collection()
            else:
                print("Please login first.")
        elif choice == "12":
            lib.suggest_personalized_book()
        elif choice == "13":
            if user:
                lib.logout()
                user = None  # Kullanıcı oturumu kapatıldı
            else:
                print("Please login first.")
        elif choice == "q":
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()



*** MENU ***

*** Main options ***
1) List Books
2) Add Book
3) Remove Book
4) Filter Books
5) Analyze Books
6) I'm Feeling Lucky
*** Login and registration ***
7) Register
8) Login
*** Special Options for Registered Users ***
9) Add books to user's private collection
10) Remove books from user's private collection
11) List books in user's private collection
12) Suggest Personalized Book
13) Logout
q) Exit
Enter your choice: 1
Listing books:
1. Book Title: Harry Potter ve Felsefe Taşı, Author: J.K. Rowling
2. Book Title: Harry Potter ve Sırlar Odası, Author: J.K. Rowling
3. Book Title: Harry Potter ve Azkaban Tutsağı, Author: J.K. Rowling
4. Book Title: Harry Potter ve Ateş Kadehi, Author: J.K. Rowling
5. Book Title: Harry Potter ve Zümrüdüanka Yoldaşlığı, Author: J.K. Rowling
6. Book Title: Harry Potter ve Melez Prens, Author: J.K. Rowling
7. Book Title: Harry Potter ve Ölüm Yadigarları, Author: J.K. Rowling
8. Book Title: Da Vinci Şifresi, Author: Dan Brown
9. Book Title: Melekler v

KeyboardInterrupt: Interrupted by user